# Описание

Выполним небольшой проект на усвоенные знания в модуле 3 Aggregates

Для выполнения задания создали набор данных которых хранится в файле *../files/ad_clicks.csv*

Проект будет заключаться в проведении **A/B Тестирования**

In [21]:
import pandas as pd

ad_click = pd.read_csv('../files/ad_clicks.csv')

print(ad_click.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


In [22]:
# Давайте посмотрим из какого источника приходит больше всего пользователей (utm_source)

the_most_clicks_from_source = ad_click.groupby('utm_source').user_id.count().reset_index()

print(the_most_clicks_from_source.sort_values(by='user_id', ascending=False))

  utm_source  user_id
2     google      680
1   facebook      504
0      email      255
3    twitter      215


In [23]:
# создайте новую колонку которая будет содержать булевые значения в зависимости от наличия данных в колонке ad_click_timestamp
# дайте новое имя колонки is_click

ad_click['is_click'] = ad_click.ad_click_timestamp.notna()

print(ad_click.head())


                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
1                NaN                  B     False  
2                NaN                  A     False  
3                NaN                  B     False  
4                NaN                  B     False  


In [33]:
# Теперь рассчитайте процент кликнувших пользователей по рекламе в разрезе каждого источника трафика

# для начала аггрегируем значения по двум колонкам utm_source и is_click

clicks_by_source = ad_click.groupby(['utm_source','is_click']).user_id.count().reset_index()

print(clicks_by_source)

# теперь сформируем пивотную таблицу

clicks_pivot = clicks_by_source.pivot(
    index='utm_source',
    columns='is_click',
    values='user_id'
).reset_index()

print()
print(clicks_pivot)

# добавим дополнительную колонку значения которой будет результат вычисления процента кликнувших пользователей
# для рассчета процента используется формула True * 100 / (False + True)

clicks_pivot['percent_clicked'] = clicks_pivot.iloc[:,2] * 100 / (clicks_pivot.iloc[:,2] + clicks_pivot.iloc[:,1])

print()
print(clicks_pivot)


  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66

is_click utm_source  False  True  percent_clicked
0             email    175    80        31.372549
1          facebook    324   180        35.714286
2            google    441   239        35.147059
3           twitter    149    66        30.697674


In [34]:
# также в наборе данных содержится колонка experimental_group которая показывает какая реклама была показа А или B
# давайте посмотрим дествительно ли реклама A и B была показана одинаковому кол-ву пользователей сайта

print(ad_click.groupby('experimental_group').user_id.count().reset_index())

  experimental_group  user_id
0                  A      827
1                  B      827


In [36]:
# Используя ранее созданную колонку is_click определите на какую рекламу нажимали чаще на А или В

print(ad_click.groupby(['experimental_group','is_click']).user_id.count().reset_index())

  experimental_group  is_click  user_id
0                  A     False      517
1                  A      True      310
2                  B     False      572
3                  B      True      255


In [37]:
# Создайте два отдельных dataframe a_clicks и b_clicks
a_clicks = ad_click[ad_click.experimental_group == 'A']
b_clicks = ad_click[ad_click.experimental_group == 'B']

In [41]:
# Для каждой группы расчитайте процент пользователей которые нажимали на рекламу в разрезе дней
a_pivot = a_clicks.groupby(['day', 'is_click']).user_id.count().reset_index().pivot(
    index='day',
    columns='is_click',
    values='user_id'
).reset_index()

b_pivot = b_clicks.groupby(['day', 'is_click']).user_id.count().reset_index().pivot(
    index='day',
    columns='is_click',
    values='user_id'
).reset_index()

a_pivot['percent_click'] = a_pivot.iloc[:,2] * 100 / (a_pivot.iloc[:,2] + a_pivot.iloc[:,1])
b_pivot['percent_click'] = b_pivot.iloc[:,2] * 100 / (b_pivot.iloc[:,2] + b_pivot.iloc[:,1])


print(a_pivot)
print()
print(b_pivot)

is_click            day  False  True  percent_click
0            1 - Monday     70    43      38.053097
1           2 - Tuesday     76    43      36.134454
2         3 - Wednesday     86    38      30.645161
3          4 - Thursday     69    47      40.517241
4            5 - Friday     77    51      39.843750
5          6 - Saturday     73    45      38.135593
6            7 - Sunday     66    43      39.449541

is_click            day  False  True  percent_click
0            1 - Monday     81    32      28.318584
1           2 - Tuesday     74    45      37.815126
2         3 - Wednesday     89    35      28.225806
3          4 - Thursday     87    29      25.000000
4            5 - Friday     90    38      29.687500
5          6 - Saturday     76    42      35.593220
6            7 - Sunday     75    34      31.192661
